In [ ]:
# Configuración base para reproducibilidad
GLOBAL_SEED = 42
DATASET_ID = "<por_definir>"


# Guía rápida de la Tarea 1

Este notebook crea archivos de texto con los puntos clave del enunciado de la tarea, para usarlos como recordatorio durante el desarrollo del proyecto. Ejecuta la celda siguiente para generar la carpeta `guia_tarea/` con un archivo por cada hito requerido.


In [ ]:
from pathlib import Path

pasos = {
    "01_ETP_y_framing.txt": """1. E–T–P y framing\n- Describe la Experiencia (E), la Tarea (T) y el Performance/criterio (P).\n- Define el framing principal como clasificación binaria de `purchase` con salida probabilística.\n- Justifica si usarás tasas agregadas y la decisión de negocio basada en umbrales.""",
    "02_metricas_y_perdida.txt": """2. Métricas y pérdida\n- Usa log-loss (entropía cruzada) como pérdida principal.\n- Reporta AUC y Brier score.\n- Explica por qué MSE no es adecuado como objetivo principal y relaciónalo con máxima verosimilitud.""",
    "03_validacion_y_capacidad.txt": """3. Diseño de validación y control de capacidad\n- Define particiones train/valid/test (70/15/15) o k-fold para escoger hiperparámetros.\n- Reentrena con los mejores hiperparámetros antes del test.\n- Controla capacidad con regularización L2 (parámetro C) y grafica curvas train/valid vs complejidad o learning curves.\n- Explica el trade-off sesgo–varianza.""",
    "04_preprocesamiento.txt": """4. Preprocesamiento\n- Explora el dataset para identificar duplicados, variables irrelevantes o leakage.\n- Define codificación de variables categóricas, escalamiento y manejo de outliers o nulos según corresponda.""",
    "05_modelado_predictivo.txt": """5. Modelado predictivo\n- Entrena al menos dos modelos de clasificación (ej. regresión logística, árbol, random forest, XGBoost).\n- Compara su desempeño inicial.""",
    "06_evaluacion.txt": """6. Evaluación\n- Reporta métricas de clasificación: accuracy, precision, recall, F1-score y AUC-ROC.""",
    "07_discusion_resultados.txt": """7. Discusión de resultados\n- Destaca variables relevantes para los modelos.\n- Justifica columnas excluidas (especialmente leaks).\n- Propón insights accionables para la empresa.""",
    "08_politica_operativa.txt": """8. Política operativa y sensibilidad\n- Formula una regla clara: contactar/ofrecer si la probabilidad estimada supera el umbral t.\n- Analiza sensibilidad (ej. utilidad esperada por umbral) y discute implicancias.""",
    "09_riesgos_y_mitigacion.txt": """9. Riesgos y mitigación\n- Identifica al menos tres riesgos: leakage, sesgo de muestreo, shift temporal/segmento.\n- Propón mitigaciones (auditoría de variables, validación por segmento o fuera de tiempo, calibration, A/B).""",
    "10_resultados_y_conclusiones.txt": """10. Resultados y conclusiones\n- Resume hallazgos clave: saturación, desempeño en test, umbral recomendado.\n- Indica cómo debe operar la empresa con el modelo final.""",
    "extras_opcionales.txt": """Extras opcionales\n- Mostrar efectos de usar `leak_after_offer` para evidenciar data leakage.\n- Calcular métricas de negocio (ej. expected profit) usando `unit_margin_if_buy`.\n- Comparar `discount` numérico vs `discount_bucket`.""",
    "requisitos_reproducibilidad.txt": """Formato y reproducibilidad\n- Declara semilla global y DATASET_ID en la primera celda.\n- Fija semillas para NumPy/sklearn y repórtalas.\n- Reporta versiones de librerías y hash SHA-256 del CSV.\n- Trabaja sólo con el dataset entregado, sin regenerar datos.\n- Entrega resultados específicos de tu dataset.""",
}

destino = Path('guia_tarea')
destino.mkdir(exist_ok=True)
for nombre, contenido in pasos.items():
    ruta = destino / nombre
    ruta.write_text(contenido.strip() + '
', encoding='utf-8')

sorted_archivos = sorted(p.name for p in destino.iterdir())
print(f'Se crearon/actualizaron {len(sorted_archivos)} archivos en {destino.resolve()}')
for archivo in sorted_archivos:
    print(f'- {archivo}')
